<h1 style="color:#191970"> Gathering Data </h1>

<h2 style="color:purple"> Import Necessary Libraries and Modules </h2>

In [41]:
import info_grabber as grabber
import data_cleaner as dc
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div style="display:inline-block"><img src="SS/yelp.jpeg" position="left" width=50/></div>
<div style="display:inline-block; padding:1px"> <h3> Gathering data from Yelp using Yelp Fusion API</h3></div><br><br>
We want to collect the business data from api_data and create an initial pandas dataframe to explore the data. 

In [916]:
# Take a look at yelp_api_grabber to dig deeper into how the TutoringDataGrabber class works

# Instatiate the TutoringDataGrabber to collect the data
data_grabber = grabber.TutoringDataGrabber(["Norwalk, CT", "Rochester, NY", "NYC", "Jersey City, NJ", "Newark, NJ", "New Haven, CT"])

# collect the data for certain locations in the tristate area
api_data_info = data_grabber.gather_tutoring_data()

<h3> Initial view of raw api data </h3>
<br>
<i>You can skip this section if you have read and understood the methods in the yelp_api_grabber document. However, if you are insterested in my process of figuring out what information is important peek below.</i> 

In [917]:
# First view the length of the gathered data to determine how many data points we have collected.
len(api_data_info)

270

In [919]:
def compare(info, item_to_check, dic_key=None, br=True):
    call = str(info)
    if info == "keys":
        info = lambda x: x.keys()
    elif info == "values":
        info = lambda x: x.values()
    if dic_key == None:
        comparisons = [info(item_to_check[0])]
    else:
        comparisons = [info(item_to_check[0][dic_key])]
    for num, data in enumerate(item_to_check[1:]):
        if dic_key != None:
            item = info(data[dic_key])
        else:
            item = info(data)
        if comparisons[0] != item:
            comparisons.append((num, item))
            if br:
                return f"Original: {comparisons[0]}\n Different from original: {comparisons[1:]}"
    if not br:
        return f"Original: {comparisons[0]}\n Different from original: {comparisons[1:]}"
                
    return f"No difference! All {call} are {comparisons[0]}."

In [920]:
# Take a look the type of the elements in api_data
print(compare(type, api_data_info))

No difference! All <class 'type'> are <class 'dict'>.


In [921]:
# Each element in api_data is a dictionary, what are the important keys and see if 
print(compare("keys", api_data_info))

Original: dict_keys(['businesses', 'total', 'region'])
 Different from original: [(51, dict_keys(['error']))]


In [924]:
print(compare("keys", api_data_info, br=False))

Original: dict_keys(['businesses', 'total', 'region'])
 Different from original: [(51, dict_keys(['error'])), (52, dict_keys(['error'])), (53, dict_keys(['error'])), (54, dict_keys(['error'])), (55, dict_keys(['error'])), (56, dict_keys(['error'])), (57, dict_keys(['error'])), (58, dict_keys(['error'])), (59, dict_keys(['error'])), (60, dict_keys(['error'])), (61, dict_keys(['error'])), (62, dict_keys(['error'])), (63, dict_keys(['error'])), (64, dict_keys(['error'])), (65, dict_keys(['error'])), (66, dict_keys(['error'])), (67, dict_keys(['error'])), (68, dict_keys(['error'])), (69, dict_keys(['error'])), (70, dict_keys(['error'])), (71, dict_keys(['error'])), (72, dict_keys(['error'])), (73, dict_keys(['error'])), (74, dict_keys(['error'])), (75, dict_keys(['error'])), (76, dict_keys(['error'])), (77, dict_keys(['error'])), (78, dict_keys(['error'])), (79, dict_keys(['error'])), (80, dict_keys(['error'])), (81, dict_keys(['error'])), (82, dict_keys(['error'])), (83, dict_keys(['error

In [925]:
api_data_info[189]

{'error': {'code': 'VALIDATION_ERROR',
  'description': '1050 is greater than the maximum of 1000',
  'field': 'offset',
  'instance': 1050}}

Every item in our gathered list are all the same type and but some have an error because we maxed out on the yelp api call. We are interested in the values that pair with the business key, so we must account for the missing key in our cleaning function. 

In [928]:
# Let's see the data type for the businesses key.
type(api_data_info[0]["businesses"])

list

We also see that the type for the value in for the business key is a list. We should account for list of different lengths in our cleaning function.

In [929]:
# Let's see the data type of an item in the values for the businesses key.
type(api_data_info[0]["businesses"][0])

dict

Lastly, we see that the type for an item in the value list for the business key is a dict. We continue this frame of thinking as we prepare our function for cleaning the data. Thinking about potential missing dict keys and mismatching list lengths.

<h3> Preprocessing: Initial clean of the api data </h3>

We'll use the functions in the data_cleaner.py script to create make columns out of information that ae stored in list and dictionaries, drop companies that are not tutoring companies, drop duplicates, and drop companies that are not in NY, CT, or NJ.

In [2386]:
# Initialize the cleaner by giving it the api_data
cleaner = dc.TutoringDataCleaner(api_data_info)

# convert the business data into dataframe
df = cleaner.parse(api_data_info)
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,mcat-king-new-york,"[{'alias': 'privatetutors', 'title': 'Private ...","{'latitude': 40.752136, 'longitude': -73.990684}",(212) 220-1538,62800.199569,oS0Ww3Nnq-gMlN8C_7zkYw,https://s3-media4.fl.yelpcdn.com/bphoto/Swj-5Q...,False,"{'address1': '213 W 35th St', 'address2': 'Ste...",MCAT KING,+12122201538,NaN,5.0,11,[],https://www.yelp.com/biz/mcat-king-new-york?ad...
1,manhattan-elite-prep-new-york-3,"[{'alias': 'tutoring', 'title': 'Tutoring Cent...","{'latitude': 40.754169, 'longitude': -73.97982}",(888) 215-6269,61952.737470,KNlRkqWwok9Sohm3Ejzfxw,https://s3-media4.fl.yelpcdn.com/bphoto/O0If9p...,False,"{'address1': '521 5th Ave', 'address2': 'Fl 17...",Manhattan Elite Prep,+18882156269,NaN,4.0,11,[],https://www.yelp.com/biz/manhattan-elite-prep-...
2,cates-tutoring-new-york-2,"[{'alias': 'privatetutors', 'title': 'Private ...","{'latitude': 40.75081, 'longitude': -73.97354}",(212) 359-4208,61797.384788,ryqawESyMqitMuZCmsxGcQ,https://s3-media1.fl.yelpcdn.com/bphoto/Ze2Xgm...,False,"{'address1': '205 E 42nd St', 'address2': 'Fl ...",CATES Tutoring,+12123594208,NaN,2.5,8,[],https://www.yelp.com/biz/cates-tutoring-new-yo...
3,simply-brilliant-manhattan,"[{'alias': 'testprep', 'title': 'Test Preparat...","{'latitude': 40.7593941, 'longitude': -73.9697...",(877) 246-1711,60933.364597,8o7o1yQ-3dnXlFQk4PhsmQ,https://s3-media2.fl.yelpcdn.com/bphoto/XL3isM...,False,"{'address1': None, 'address2': None, 'address3...",Simply Brilliant,+18772461711,NaN,5.0,4,[],https://www.yelp.com/biz/simply-brilliant-manh...
4,sheptin-tutoring-group-chappaqua-3,"[{'alias': 'tutoring', 'title': 'Tutoring Cent...","{'latitude': 41.1595399, 'longitude': -73.764855}",(914) 232-3743,29444.437821,IBL3Y39_I4rxpFmBQ2zjQg,https://s3-media3.fl.yelpcdn.com/bphoto/Qz_3rz...,False,"{'address1': '175 King St', 'address2': None, ...",Sheptin Tutoring Group,+19142323743,NaN,5.0,3,[],https://www.yelp.com/biz/sheptin-tutoring-grou...


In [2389]:
# initial view of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7479 entries, 0 to 7478
Data columns (total 16 columns):
alias            7479 non-null object
categories       7479 non-null object
coordinates      7479 non-null object
display_phone    7479 non-null object
distance         7479 non-null float64
id               7479 non-null object
image_url        7479 non-null object
is_closed        7479 non-null bool
location         7479 non-null object
name             7479 non-null object
phone            7479 non-null object
price            386 non-null object
rating           7479 non-null float64
review_count     7479 non-null int64
transactions     7479 non-null object
url              7479 non-null object
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 883.8+ KB


We have only 386 rows with prices. We would drop this column for this project. In the future, it might be a good idea to look at the price as a feature, but would need to find data from another source.

In [2390]:
# this method gives us a view of the type of each feature
cleaner.view_item_type(df).head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'bool'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'list'>,<class 'str'>
1,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'bool'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'list'>,<class 'str'>
2,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'bool'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'list'>,<class 'str'>
3,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'bool'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'list'>,<class 'str'>
4,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'bool'>,<class 'dict'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'list'>,<class 'str'>


We have some list and dictionaries in the dataframe that needs to be broken down into separate features.

In [2393]:
# this method finds the features of a certain type class
cleaner.columns_to_break_down(df)
# we are looking for lists and dictionaries

<class 'list'>
['categories' 'transactions'] 

<class 'dict'>
['coordinates' 'location'] 



array(['categories', 'coordinates', 'location', 'transactions'],
      dtype=object)

In [2403]:
# create a copy of the dataframe with the dictionary and lists items as their own columns
df_tri_state = cleaner.collapse(df)
print(df_tri_state.shape)
df_tri_state.head()

(7479, 28)


,display_phone,id,is_closed,name,phone,price,rating,review_count,url,location_0,...,transactions_2,coordinates,alias_0,title_0,alias_1,title_1,alias_2,title_2,alias_3,title_3
0,(212) 220-1538,oS0Ww3Nnq-gMlN8C_7zkYw,False,MCAT KING,+12122201538,NaN,5.0,11,https://www.yelp.com/biz/mcat-king-new-york?ad...,213 W 35th St,...,NaN,"(40.752136, -73.990684)",privatetutors,Private Tutors,testprep,Test Preparation,NaN,NaN,NaN,NaN
1,(888) 215-6269,KNlRkqWwok9Sohm3Ejzfxw,False,Manhattan Elite Prep,+18882156269,NaN,4.0,11,https://www.yelp.com/biz/manhattan-elite-prep-...,521 5th Ave,...,NaN,"(40.754169, -73.97982)",tutoring,Tutoring Centers,privatetutors,Private Tutors,testprep,Test Preparation,NaN,NaN
2,(212) 359-4208,ryqawESyMqitMuZCmsxGcQ,False,CATES Tutoring,+12123594208,NaN,2.5,8,https://www.yelp.com/biz/cates-tutoring-new-yo...,205 E 42nd St,...,NaN,"(40.75081, -73.97354)",privatetutors,Private Tutors,testprep,Test Preparation,careercounseling,Career Counseling,NaN,NaN
3,(877) 246-1711,8o7o1yQ-3dnXlFQk4PhsmQ,False,Simply Brilliant,+18772461711,NaN,5.0,4,https://www.yelp.com/biz/simply-brilliant-manh...,None,...,NaN,"(40.7593941, -73.9697795)",testprep,Test Preparation,privatetutors,Private Tutors,tutoring,Tutoring Centers,NaN,NaN
4,(914) 232-3743,IBL3Y39_I4rxpFmBQ2zjQg,False,Sheptin Tutoring Group,+19142323743,NaN,5.0,3,https://www.yelp.com/biz/sheptin-tutoring-grou...,175 King St,...,NaN,"(41.1595399, -73.764855)",tutoring,Tutoring Centers,privatetutors,Private Tutors,testprep,Test Preparation,NaN,NaN


Find all the unique titles that are tutoring related. Tutoring related are titles that include the word "Tutor", "Education", "Community", "Camps", "Preschool", "Prep", "Homework", "Test"

In [2405]:
# all of the unique titles
arr_comb_all = cleaner.get_unique("title", df_tri_state) 
arr_comb_all.shape

(340,)

We have 340 different titles.

In [2415]:
# gives a zero or one to all tutoring related titles and gives a count of how many titles are tutoring related
df_tri_state1 = cleaner.title_encoding(df_tri_state)
print(df_tri_state1.shape)
df_tri_state1.head()

(7479, 33)


,display_phone,id,is_closed,name,phone,price,rating,review_count,url,location_0,...,title_1,alias_2,title_2,alias_3,title_3,title_0_code,title_1_code,title_2_code,title_3_code,counts
0,(212) 220-1538,oS0Ww3Nnq-gMlN8C_7zkYw,False,MCAT KING,+12122201538,nan,5.0,11,https://www.yelp.com/biz/mcat-king-new-york?ad...,213 W 35th St,...,Test Preparation,nan,nan,nan,nan,1,1,0,0,2
1,(888) 215-6269,KNlRkqWwok9Sohm3Ejzfxw,False,Manhattan Elite Prep,+18882156269,nan,4.0,11,https://www.yelp.com/biz/manhattan-elite-prep-...,521 5th Ave,...,Private Tutors,testprep,Test Preparation,nan,nan,1,1,1,0,3
2,(212) 359-4208,ryqawESyMqitMuZCmsxGcQ,False,CATES Tutoring,+12123594208,nan,2.5,8,https://www.yelp.com/biz/cates-tutoring-new-yo...,205 E 42nd St,...,Test Preparation,careercounseling,Career Counseling,nan,nan,1,1,0,0,2
3,(877) 246-1711,8o7o1yQ-3dnXlFQk4PhsmQ,False,Simply Brilliant,+18772461711,nan,5.0,4,https://www.yelp.com/biz/simply-brilliant-manh...,None,...,Private Tutors,tutoring,Tutoring Centers,nan,nan,1,1,1,0,3
4,(914) 232-3743,IBL3Y39_I4rxpFmBQ2zjQg,False,Sheptin Tutoring Group,+19142323743,nan,5.0,3,https://www.yelp.com/biz/sheptin-tutoring-grou...,175 King St,...,Private Tutors,testprep,Test Preparation,nan,nan,1,1,1,0,3


In [2291]:
# drop all rows(companies) that have a count of 0 for tutoring related titles
df_tri_state1 = cleaner.find_tutoring(df_tri_state1)
print(df_tri_state1.shape)
df_tri_state1.head()

(3714, 30)


,display_phone,id,is_closed,name,phone,price,rating,review_count,url,location_0,...,title_1,alias_2,title_2,alias_3,title_3,title_0_code,title_1_code,title_2_code,title_3_code,counts
0,(212) 220-1538,oS0Ww3Nnq-gMlN8C_7zkYw,False,MCAT KING,+12122201538,NaN,5.0,11,https://www.yelp.com/biz/mcat-king-new-york?ad...,213 W 35th St,...,Test Preparation,NaN,NaN,NaN,NaN,1,1,0,0,2
1,(888) 215-6269,KNlRkqWwok9Sohm3Ejzfxw,False,Manhattan Elite Prep,+18882156269,NaN,4.0,11,https://www.yelp.com/biz/manhattan-elite-prep-...,521 5th Ave,...,Private Tutors,testprep,Test Preparation,NaN,NaN,1,1,1,0,3
2,(212) 359-4208,ryqawESyMqitMuZCmsxGcQ,False,CATES Tutoring,+12123594208,NaN,2.5,8,https://www.yelp.com/biz/cates-tutoring-new-yo...,205 E 42nd St,...,Test Preparation,careercounseling,Career Counseling,NaN,NaN,1,1,0,0,2
3,(877) 246-1711,8o7o1yQ-3dnXlFQk4PhsmQ,False,Simply Brilliant,+18772461711,NaN,5.0,4,https://www.yelp.com/biz/simply-brilliant-manh...,None,...,Private Tutors,tutoring,Tutoring Centers,NaN,NaN,1,1,1,0,3
4,(914) 232-3743,IBL3Y39_I4rxpFmBQ2zjQg,False,Sheptin Tutoring Group,+19142323743,NaN,5.0,3,https://www.yelp.com/biz/sheptin-tutoring-grou...,175 King St,...,Private Tutors,testprep,Test Preparation,NaN,NaN,1,1,1,0,3


We dropped half of the companies we gathered because they were not tutoring related.

In [2292]:
# drop any duplicates
df_biz_tri = df_tri_state1.drop_duplicates().reset_index().drop(["index"], axis=1)
print(df_biz_tri.shape)
df_biz_tri.head()

(1177, 30)


,display_phone,id,is_closed,name,phone,price,rating,review_count,url,location_0,...,title_1,alias_2,title_2,alias_3,title_3,title_0_code,title_1_code,title_2_code,title_3_code,counts
0,(212) 220-1538,oS0Ww3Nnq-gMlN8C_7zkYw,False,MCAT KING,+12122201538,NaN,5.0,11,https://www.yelp.com/biz/mcat-king-new-york?ad...,213 W 35th St,...,Test Preparation,NaN,NaN,NaN,NaN,1,1,0,0,2
1,(888) 215-6269,KNlRkqWwok9Sohm3Ejzfxw,False,Manhattan Elite Prep,+18882156269,NaN,4.0,11,https://www.yelp.com/biz/manhattan-elite-prep-...,521 5th Ave,...,Private Tutors,testprep,Test Preparation,NaN,NaN,1,1,1,0,3
2,(212) 359-4208,ryqawESyMqitMuZCmsxGcQ,False,CATES Tutoring,+12123594208,NaN,2.5,8,https://www.yelp.com/biz/cates-tutoring-new-yo...,205 E 42nd St,...,Test Preparation,careercounseling,Career Counseling,NaN,NaN,1,1,0,0,2
3,(877) 246-1711,8o7o1yQ-3dnXlFQk4PhsmQ,False,Simply Brilliant,+18772461711,NaN,5.0,4,https://www.yelp.com/biz/simply-brilliant-manh...,None,...,Private Tutors,tutoring,Tutoring Centers,NaN,NaN,1,1,1,0,3
4,(914) 232-3743,IBL3Y39_I4rxpFmBQ2zjQg,False,Sheptin Tutoring Group,+19142323743,NaN,5.0,3,https://www.yelp.com/biz/sheptin-tutoring-grou...,175 King St,...,Private Tutors,testprep,Test Preparation,NaN,NaN,1,1,1,0,3


We will remove any companies that snuck in that are not in the tri-state area(NY, NJ, CT).

In [2293]:
df_biz_tri_only = df_biz_tri[df_biz_tri.location_6.apply(lambda x: x in ["CT", "NY", "NJ"])].reset_index().drop(["index"], axis=1)
df_biz_tri_only.shape

(1167, 30)

In [2294]:
# save the df in a json file
df_biz_tri_only.to_json("tristatedata.json")

### Getting area data from areavibes.com 

<img src="SS/Screen Shot 2019-05-14 at 4.37.31 PM.png" width=500/>
<p style="font-size:8px; float:right">Source: areavibes.com</p>
<br>
For each company, we will collect their zipcode and scrape areavibes for the area data using beautiful and selenium.

In [2297]:
# create an instance of the tutoring scraper from info_grabber.py
scraper = grabber.TutoringScraper("/Users/flatironschool/Downloads/chromedriver")

# get all of the unique zipcodes in the dataframe with yelp api data
# the zipcodes are found in the location_4 column
zipcodes = df_biz_tri_only.location_4.unique()

# create a dataframe with the areavibes data
zipcode_df = scraper.get_multi_area_data(zipcodes)

In [2301]:
print(zipcode_df.shape)
zipcode_df.head()

(290, 9)


,Amenities,Cost of Living,Crime,Employment,Housing,Livability,Schools,Weather,zipcode
10001,A+,F,F,A+,F,70,A+,C-,10001
10175,A+,F,F,A+,F,64,A+,C-,10175
10017,A+,F,F,A+,F,64,A+,C-,10017
10022,A+,F,F,A+,F,64,A+,C-,10022
10514,A+,F,A+,A+,F,77,A+,D+,10514


<h3> Preprocessing: Initial clean of the areavibes.com data </h3>

We want to convert the letter grades into a numerical value. F - A+ is mapped to 1 - 13.

In [2307]:
# create an instance of the area data cleaner
acleaner = dc.AreaDataCleaner(df_biz_tri_only)

In [2305]:
# use a cleaning method from the cleaning script to create a copy of the dataframe with only numerical values
livingdf = acleaner.clean_living_data(df_biz_tri_only ,zipcode_df)
livingdf.head()

,Amenities,Cost of Living,Crime,Employment,Housing,Livability,Schools,Weather,zipcode
0,13,1,1,13,1,70,13,5,10001
1,13,1,1,13,1,64,13,5,10175
2,13,1,1,13,1,64,13,5,10017
3,13,1,1,13,1,64,13,5,10022
4,13,1,13,13,1,77,13,4,10514


In [2304]:
# save areavibe data
livingdf.to_json("areavibedata.json")

### Gathering demographic data from  movoto.com for each zipcode
<img src="SS/Screen Shot 2019-05-14 at 8.36.59 PM.png" width=800/>
<p style="font-size:8px; float:right">Source: movoto.com</p>
<br><br>
For each company, we will use the state and zipcode to collect the demographic data from movoto.com.

In [2311]:
# scrapes the demographic information from movoto.com
dem = scraper.get_all_area_dem(df_biz_tri_only)
dem.head()

,"$1,000 to $1,499","$1,500 to $1,999","$150,000 or More",$2000 or More,"$30,000 to $74,999",$600 to $999,"$75,000 to $149,999",1 Bedroom,1 Person,1 Room,...,Public Administration,Public Transportation,Renter Occupied,Retail/Wholesale,Taxi,Transportation/Warehousing,Unemployed,Walk or by Bicycle,Work at Home,zipcode
0,0.113,0.088,0.322,0.482,0.2,0.15,0.23,0.458,0.536,0.156,...,0.019,0.416,0.728,0.099,0.036,0.027,0.059,0.429,0.057,10001
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10175
0,779 (12.2%),"1,439 (22.6%)","3,531 (34.8%)","3,645 (57.2%)","2,220 (21.9%)",286 (4.5%),"2,925 (28.8%)","5,758 (44.8%)","6,184 (60.9%)","3,135 (24.4%)",...,535 (4.4%),"4,323 (35.9%)","6,417 (63.2%)","1,531 (12.7%)",315 (2.6%),104 (0.9%),416 (2.7%),"5,824 (48.4%)",805 (6.7%),10017
0,780 (8.7%),"2,245 (25%)","7,775 (43.5%)","5,045 (56.1%)","3,286 (18.4%)",679 (7.6%),"4,490 (25.1%)","11,378 (48.4%)","9,966 (55.7%)","2,908 (12.4%)",...,460 (2.4%),"7,155 (38.6%)","9,198 (51.4%)","1,576 (8.3%)",831 (4.5%),150 (0.8%),896 (3.2%),"6,667 (36%)","1,663 (9%)",10022
0,63 (24.5%),77 (30%),"2,723 (67.8%)",68 (26.5%),393 (9.8%),0 (0%),697 (17.3%),65 (1.5%),483 (12%),0 (0%),...,78 (1.3%),"1,909 (32.6%)",257 (6.4%),481 (8.2%),0 (0%),48 (0.8%),151 (1.6%),144 (2.5%),681 (11.6%),10514


<h3> Preprocessing: Initial clean of the movoto.com dataframe </h3>

We want to change and lowercase the names of the columns, drop columns about employment types, and make the percentage datapoints within the parantheses into decimals.

In [2312]:
# create a copy of the dataframe cleaned as mentioned above
demographics = acleaner.clean_area_dem(dem)
demographics.head()

,1k_to_1499_rent,"1.5k_to_1,999_rent",150k_plus_salary,2k_plus_rent,"30k_to_74,999_salary",600_to_999_rent,"75k_to_149,999_salary",1_person_household,age_range_10_to_17,age_range_18_to_24,...,not_in_labor_force,owner_occupied_home,private_vehicle,public_transportation,renter_occupied_home,taxi,unemployed,walk_or_bicycle,work_from_home,zipcode
0,0.113,0.088,0.322,0.482,0.2,0.15,0.23,0.536,0.054,0.147,...,0.294,0.272,0.046,0.416,0.728,0.036,0.059,0.429,0.057,10001
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10175
2,0.122,0.226,0.348,0.572,0.219,0.045,0.288,0.609,0.006,0.078,...,0.191,0.368,0.054,0.359,0.632,0.026,0.027,0.484,0.067,10017
3,0.087,0.25,0.435,0.561,0.184,0.076,0.251,0.557,0.016,0.046,...,0.294,0.486,0.114,0.386,0.514,0.045,0.032,0.36,0.09,10022
4,0.245,0.3,0.678,0.265,0.098,0,0.173,0.12,0.153,0.062,...,0.351,0.936,0.531,0.326,0.064,0,0.016,0.025,0.116,10514


In [2313]:
# save demographic dataframe
demographic.to_json("dgraphics.json")

### Getting Review Data From Yelp
<img src="Tutoring_Company_Project/SS/Screen Shot 2019-05-14 at 11.01.48 PM.png" width=500/>
<p style="font-size:8px; float:right">Source: yelp.com</p>
<br>
We will web scrape yelp.com for information not available through the yelp api, mainly the user reviews of the tutoring companies in the dataframe. 

In [1031]:
# scrape yelp.com and get reviews from all users
dfrev = scraper.get_all_reviews(df_biz_tri_only)

0


In [2338]:
# preview of the data collected
dfrev[:5]

[{'MCAT KING': {'User': [['Marina A.', 'Staten Island, NY'],
    ['Nazia R.', 'JAMAICA, NY'],
    ['Becca D.', 'Brooklyn, NY'],
    ['Lev S.', 'New York, NY'],
    ['Melvin P.', 'New York, NY'],
    ['Michelle S.', 'Los Angeles, CA'],
    ['Jacob Z.', 'Great Neck, NY'],
    ['Nicole Z.', 'Great Neck, NY'],
    ['Michelle L.', 'Staten Island, NY'],
    ['Justin R.', 'Plainview, NY'],
    ['Ashley M.', 'New York, NY']],
   'User_Stat': [['108 friends', '1 review'],
    ['36 friends', '13 reviews'],
    ['179 friends', '4 reviews', '4 photos'],
    ['306 friends', '25 reviews', '4 photos'],
    ['0 friends', '13 reviews'],
    ['0 friends', '8 reviews', '1 photo'],
    ['0 friends', '6 reviews', '8 photos'],
    ['0 friends', '5 reviews', '5 photos'],
    ['280 friends', '3 reviews'],
    ['18 friends', '7 reviews'],
    ['0 friends', '5 reviews']],
   'Rev_Date': ['2018-12-08',
    '2018-12-29',
    '2018-12-09',
    '2018-04-20',
    '2018-12-07',
    '2018-08-28',
    '2017-08-27',
   

<h3> Preprocessing: Initial clean of the yelp.com review dataframe </h3>

We want to convert the list into a dataframe. We will change the initial name of each business to include a number at the end to distinguish the difference between to businesses with the same name. We will then have two dataframes, one with each user review as a row and another with the combination of reviews for each business.

In [18]:
# create an instance of the review cleaner class
review_cleaner = dc.ReviewDataCleaner(dfrev)

# create a clean dataframe with the reviews
cleaned_review_df = review_cleaner.clean_all_reviews(dfrev)
print(cleaned_review_df.shape)
cleaned_review_df.head()

In [2453]:
# save the reviews
cleaned_review_df.to_json("user_reviews.json")

In [44]:
cleaned_review_df = pd.read_json("user_reviews.json")

In [45]:
# create a copy of the dataframe with the names of the businesses altered
# we add the same number if the name is the same and then increase the number added to the end of the business name
new_rev = review_cleaner.organize_reviews(cleaned_review_df)
print(new_rev.shape)
new_rev.head()

(11991, 9)


,business_name,review,star_rating,user_location,user_name,friend_count,review_count,photo_count,elite_status
0,MCAT KING1,Noble is THE best! I can't believe I waited so...,5,"Staten Island, NY",Marina A.,108,1,0,0
1,MCAT KING1,Noble is such an amazing tutor!! He is very pa...,5,"JAMAICA, NY",Nazia R.,36,13,0,0
10,MCAT KING1,First to Review When struggling with Organic C...,5,"New York, NY",Ashley M.,0,5,0,0
100,Stepping Stones Museum For Children2,This place is great. Just went there for the ...,5,"Yonkers, NY",Matt A.,0,4,0,0
1000,Tutors of Oxford NYC3,First to Review I needed to pass precalculus i...,5,"New York, NY",D. T.,0,4,0,0


In [46]:
# we consolidate the reviews into a collection of reviews per business
consolidated_reviews = pd.DataFrame(new_rev.groupby("business_name", sort=False).apply(lambda x: x.review.tolist()), columns=["reviews"])
print(consolidated_reviews.shape)
consolidated_reviews.head()



(3610, 1)


,reviews
business_name,
MCAT KING1,[Noble is THE best! I can't believe I waited s...
Stepping Stones Museum For Children2,[This place is great. Just went there for the...
Tutors of Oxford NYC3,[First to Review I needed to pass precalculus ...
Small World Preschool4,[Small World is a great preschool. My son has ...
YMCA of Rye NY5,[3 check-ins I've been a member of this locati...


In [47]:
# we consolidate the businesses and get the mean of the numerical values
new_rev_no_revs = review_cleaner.drop_columns(new_rev, ["review","user_location","user_name"])
consolidated_means = pd.DataFrame(new_rev_no_revs.groupby("business_name", sort=False).apply(lambda x: x.astype(float).mean()))
consolidated_means.head()



,star_rating,friend_count,review_count,photo_count,elite_status
business_name,,,,,
MCAT KING1,5.0,48.0,6.333333,0.0,0.0
Stepping Stones Museum For Children2,5.0,0.0,4.000000,0.0,0.0
Tutors of Oxford NYC3,5.0,0.0,4.000000,0.0,0.0
Small World Preschool4,1.8,0.8,36.600000,0.4,0.0
YMCA of Rye NY5,3.4,52.6,191.600000,278.2,0.2


In [48]:
# combine the the dataframe with only the reviews and the means
consolidated = pd.concat([consolidated_means, consolidated_reviews], axis=1).reset_index()
consolidated.head()

,business_name,star_rating,friend_count,review_count,photo_count,elite_status,reviews
0,MCAT KING1,5.0,48.0,6.333333,0.0,0.0,[Noble is THE best! I can't believe I waited s...
1,Stepping Stones Museum For Children2,5.0,0.0,4.000000,0.0,0.0,[This place is great. Just went there for the...
2,Tutors of Oxford NYC3,5.0,0.0,4.000000,0.0,0.0,[First to Review I needed to pass precalculus ...
3,Small World Preschool4,1.8,0.8,36.600000,0.4,0.0,[Small World is a great preschool. My son has ...
4,YMCA of Rye NY5,3.4,52.6,191.600000,278.2,0.2,[3 check-ins I've been a member of this locati...


In [49]:
# remove the number from the end of the business name

# create a column with the index + 1 as a string
consolidated["index"] = (consolidated.reset_index()["index"] + 1).astype(str)
latest = pd.DataFrame()
for _, row in consolidated.iterrows():
    # create a new dataframe similar to consolidated but with the number stripped off the end
    row.business_name = row.business_name.strip(row["index"] )
    latest = pd.concat([latest, row], axis=1, sort=False)

latest = latest.T
latest.head()

,business_name,star_rating,friend_count,review_count,photo_count,elite_status,reviews,index
0,MCAT KING,5,48,6.33333,0,0,[Noble is THE best! I can't believe I waited s...,1
1,Stepping Stones Museum For Children,5,0,4,0,0,[This place is great. Just went there for the...,2
2,Tutors of Oxford NYC,5,0,4,0,0,[First to Review I needed to pass precalculus ...,3
3,Small World Preschool,1.8,0.8,36.6,0.4,0,[Small World is a great preschool. My son has ...,4
4,YMCA of Rye NY,3.4,52.6,191.6,278.2,0.2,[3 check-ins I've been a member of this locati...,5


In [50]:
latest.shape

(3610, 8)

In [51]:
latest.to_json("consolidated_review.json")